## Preprocess HRES Data

The following scripts are used for preprocessing HRES data mainly using CDO (Climate Data Operators) and partially using NCO (netCDF Operator).

https://code.mpimet.mpg.de/projects/cdo 

https://nco.sourceforge.net/

### Step 1)

- Select `tp` (precipitation) out of the many variables for reducing the size of the NetCDF file

- Change the variable name from `tp` to a common name in other used data: `pr`

- Change the unit attribute of `pr` from `m` to `mm`

- Convert the data from `m` to `mm`

- Convert **cumulative** data into **instantaneous** using `deltat` function

- Select the first **24 hours** out of 90 hours

*the changes made in step 1 are saved in `HRES_DUMP`*

### Step 2)

- Merge all the NetCDF files generated in step 1 into one single NetCDF file using `mergetime`

- Include all the changes in the `history` attribute and in a human-readable way.

*the changes made in step 2  are saved in `HRES_PP`*


### Command guides used for CDO and NCO:

CDO:

important! when combining several CDO operators, the commands should come in inverse order.
mergetime cannot be combined with other operators.

```
                -L: Lock IO
                -O: Overwrite existing output file
                -b: Number of bits for the output (F64 64-bit floating point type)
          -selname: Select parameters by name
           -deltat: Difference between timesteps
      -seltimestep: Select timesteps
        -mergetime: Merge datasets sorted by date and time
           -chname: Change name of a variable
     -setattribute: Set attributes
             -mulc: Multiply with a constant
```

NCO: 

```
          -ncatted: Attribute editor
                -O: Overwrite existing output file
                -h: append the command to the history attribute in the output-file.
                -a: attribute modification
                ,a: append
```

In [3]:
source bashenv
rm -r $HRES_PP $HRES_DUMP $HRES_LOG
mkdir $HRES_PP $HRES_DUMP $HRES_LOG

for ncfile in $(ls $HRES_OR)
do
# Step 1)
    cdo -O -b F64 --no_history -seltimestep,1/24 -deltat -mulc,1000. -setattribute,pr@units=mm -chname,tp,pr -selname,tp  $HRES_OR/$ncfile $HRES_DUMP/$ncfile.deltat.nc >& $HRES_LOG/sample.s
done
# Step 2)
cdo -O -b F64 --no_history -mergetime $HRES_DUMP/ADAP*.nc $HRES_DUMP/HRES_PP_202007_202304_nohist.nc >& $HRES_LOG/sample1.s
ncatted -O -h -a history,global,o,c,"2022.10.21: cdo -O -b F64 -seltimestep,1/24 -deltat -mulc,1000. -setattribute,pr@units=mm -chname,tp,pr -selname,tp  cdo -O -b F64 -mergetime" $HRES_DUMP/HRES_PP_202007_202304_nohist.nc $HRES_PP/HRES_PP_202007_202304.nc
#end